In [1]:
%matplotlib inline

#writing functions that accept any number of arguments

In [5]:
def avg(first,*rest):
    return (first+sum(rest))*1.0/(1+len(rest))

avg(1,2)

1.5

In [6]:
avg(1,3,4)

2.6666666666666665

In [7]:
avg(1,4,*range(5))

2.142857142857143

#Defining anonymous or inline functions

In [10]:
add=lambda x,y:x+y
add(2,3)

5

In [11]:
add('hello','world')

'helloworld'

#making an N-Argument callable work as a callable with fewer arguments

if you need to reduce the number of arguments to a function,you should use
functools.partial().The partial() function allows you to assign fixed values
to one or more of the arguments,thus reducing the number of arguments that
need to be supplied to subsequent calls.


In [2]:
def spam(a,b,c,d):
    print a,b,c,d

In [3]:
from functools import partial
s1=partial(spam,1)  #a=1
s1(2,3,4)

1 2 3 4


In [4]:
s1(3,4,5)

1 3 4 5


In [5]:
s2=partial(spam,c=43)

In [7]:
s2(1,4,d=5)

1 4 43 5


Replacing single method classes can be turned into functions using colsures.

In [18]:
from urllib import urlopen
class UrlTemplate:
    def __init__(self,template):
        self.template=template
    def open(self,**kwargs):
        return urlopen(self.template.format_map(kwargs))

In [ ]:
yahoo=UrlTemplate('http://finance.yahoo.com/d/quotes.csv?s={names}&f={fields}')
for line in yahoo.open(names='IBM,AAPL,FB',fields='sl1c1v'):
    print line.decode('utf-8')

#the class could be replaced with a much simpler function:
def urltemplate(template):
    def opener(**kwargs):
        return urlopen(template.format_map(kwargs))
    return opener

yahoo=urltemplate('http://finance.yahoo.com/d/quotes.csv?s={names}&f={fields}')
for line in yahoo(names='IBM,APPL,FB',fields='sl1c1v'):
    print line.decode('utf-8')
    

carrying extra state with callback functions

In [20]:
def apply_async(func,args,callback):
    #computer the result
    result=func(*args)
    
    #invoke the callback with the result
    callback(result)
    

In [21]:
def print_result(result):
    print 'got:',result

def add(x,y):
    return x+y

In [22]:
apply_async(add,(2,3),callback=print_result)

got: 5


In [23]:
apply_async(add,('hello','world'),callback=print_result)

got: helloworld


One way to carray extra information ina callback is to use a bound-method instead of
a simple function.

In [24]:
class ResultHandler:
    def __init__(self):
        self.sequence=0
    def handler(self,result):
        self.sequence+=1
        print '[{}] got:{}'.format(self.sequence,result)

In [25]:
r=ResultHandler()
apply_async(add,(2,3),r.handler)

[1] got:5


In [26]:
apply_async(add,('hello','world'),r.handler)

[2] got:helloworld


As an alternative to a class,you can also use a closure to capture state.

In [33]:
def make_handler():
    sequence=[0]
    def handler(result):
        sequence[0]+=1
        print '[{}] got:{}'.format(sequence[0],result)
    return handler


In [35]:
handler=make_handler()
apply_async(add,(2,3),handler)

[1] got:5


In [36]:
apply_async(add,('hello','world'),handler)

[2] got:helloworld


Accessing variables defined inside a closure

In [37]:
def sample():
    n=[0]
    #closure function
    def func():
        print 'n=',n[0]
    
    #accessing methods for n
    def get_n():
        return n[0]
    
    def set_n(value):
        n[0]=value
    
    #Attach as function attributes
    func.get_n=get_n
    func.set_n=set_n
    return func
        

In [39]:
f=sample()
f()

n= 0


In [40]:
f.set_n(10)
f()

n= 10


In [41]:
f.get_n()

10